In [1]:
import nltk
import os
import json
import math


In [2]:
# Setup
complete_posts_as_documents = {}

files = [filename for filename in os.listdir("./comments_json") if filename.split('.')[-1] == "json"]
files = sorted(files)

complete_posts_as_documents = {key:{"comments": [], "tokens":[], "text": ""} for key in files}

In [3]:
for key in complete_posts_as_documents:
    with open(f"./comments_json/{key}","r",encoding="utf8") as f:
        data = json.load(f)
    for comment in data:
        complete_posts_as_documents[key]["comments"] = complete_posts_as_documents[key]["comments"] + [comment["text"]]
        if comment["answers"]:
            complete_posts_as_documents[key]["comments"] =complete_posts_as_documents[key]["comments"] + [element["text"] for element in comment["answers"]]

    complete_posts_as_documents[key]["text"] = " \n ".join(complete_posts_as_documents[key]["comments"])
    complete_posts_as_documents[key]["tokens"] = nltk.tokenize.word_tokenize(complete_posts_as_documents[key]["text"])
    complete_posts_as_documents[key]["token_count"] = [{token:complete_posts_as_documents[key]["tokens"].count(token) } for token in set(complete_posts_as_documents[key]["tokens"])]
    complete_posts_as_documents[key]["tf"] = {token:complete_posts_as_documents[key]["tokens"].count(token)/len(complete_posts_as_documents[key]["tokens"]) for token in set(complete_posts_as_documents[key]["tokens"]) } 

tokens = list(set([token for key in complete_posts_as_documents for token in complete_posts_as_documents[key]["tokens"]]))

docs_per_term = {term:len([1 for key in complete_posts_as_documents if term in complete_posts_as_documents[key]["tokens"]]) for term in tokens}
document_count = len(complete_posts_as_documents)

for key in complete_posts_as_documents:
    complete_posts_as_documents[key]["idf"] = \
     {term:math.log10(document_count / float(docs_per_term[term])) \
        for term in set(complete_posts_as_documents[key]["tokens"])}


In [4]:
for key in complete_posts_as_documents:
    complete_posts_as_documents[key]['tfidf'] = {item[0][0] : item[0][1] * item[1][1] for item in zip(complete_posts_as_documents[key]['tf'].items(),complete_posts_as_documents[key]['idf'].items())}

tfidfvalues = [complete_posts_as_documents[key]['tfidf'][term] for key in complete_posts_as_documents for term in complete_posts_as_documents[key]['tfidf']]
avg = sum(tfidfvalues) / len(tfidfvalues)

for key in complete_posts_as_documents:
    print(key)
    print(sorted(complete_posts_as_documents[key]["tfidf"], key=complete_posts_as_documents[key]["tfidf"].get, reverse=True)[:10])

2021-04-30_11-50-30_UTC_comments.json
['Idee', 'geg_alex', 'eva.stories', 'gespannt', 'axelossenkop', 'stonehunter_13', 'tolle', '👏', 'tolles', 'Toi']
2021-04-30_11-50-55_UTC_comments.json
['lufrcrd', 'anja_ella', 'qwierly', 'asile_87', 'navitimer11', 'vinzs_fruehstuecke', 'vermeidlich', 'chukchee_', 'bekyra', 'EIGENTLICHE']
2021-04-30_11-51-17_UTC_comments.json
['jnration_y', 'Idee', 'gespannt', 'Projekt', 'cooles', 'Kassel', 'Jana', 'torbellinita', 'Tolles', 'tolles']
2021-04-30_11-56-36_UTC_comments.json
['fotoshoot_78', 'nanne._nb', 'wil', 'mijn', 'ben', 'Interviews', 'ik', 'Mega', 'Inge', 'inge']
2021-04-30_11-57-14_UTC_comments.json
['were', 'and', 'Vaterland', 'the', 'visited', 'to', 'after', 'daughter', 'taught', 'evidence']
2021-04-30_11-57-51_UTC_comments.json
['yannick.scheid', 'teq_tec', 'jusch5039', 'kreatives_maedel', 'gespannt', 'liebchen.vintage', 'natasha_borshch', 'tuanka100', 'xngi._lala', 'SWR1']
2021-05-04_10-46-32_UTC_comments.json
['kurtle1308', 'Idee', 'Projekt'

In [5]:
sample_key = list(complete_posts_as_documents.keys())[1]

In [6]:
for sample_key in complete_posts_as_documents:
    sentences = {comment[:15]:nltk.tokenize.word_tokenize(comment.replace(".",". ").replace("  "," ")) for comment in complete_posts_as_documents[sample_key]["comments"]}
    comments_count = len(sentences)

    sent_freq = {}
    for sent in sentences:
        freq = {}
        for word in sentences[sent]:
            #lword = word.lower()
            if not word in freq:
                freq[word] = 1
                continue
            freq[word] += 1
        sent_freq[sent] = freq

    sent_tf = {sent:{word: sent_freq[sent][word] / len(sent_freq[sent]) for word in sent_freq[sent] } for sent in sent_freq}
    terms = list(set([term for sent in sent_tf for term in sent_tf[sent]]))
    sent_per_term = {}
    for sent in sentences:
        for term in terms:    
            if term in sentences[sent]:
                if term not in sent_per_term:
                    sent_per_term[term] = 1
                    continue
                sent_per_term[term] += 1

    sent_idf = {sent:{term:math.log10(comments_count / float(sent_per_term[term])) for term in sentences[sent]} for sent in sentences}
    sent_tfidf = {}
    for sent in sentences:
        sent_tfidf[sent] = {item1[0]:item1[1]*item2[1] for item1, item2 in zip(sent_tf[sent].items(),sent_idf[sent].items())}

    sent_tfidf_values = {sent: sum(sent_tfidf[sent].values())/len(sent_tfidf[sent]) for sent in sent_tfidf}
    avg = sum(sent_tfidf_values.values())/len(sent_tfidf_values.values())

    for sent in complete_posts_as_documents[sample_key]["comments"]:
        if sent_tfidf_values[sent[:15]] >= avg*2.0:
            print(sent)
    print("\n\nNextpost\n")

Schon irgendwie respektlos
LulW
🙌
👏
❤️👏
🙌🙌🙌
❤️
❤️
😍❤️
🔥🔥🔥 MEGA👏
Spannendes Projekt 🙌
👏🏻👏🏼👏🏼😍
Gradiose Idee👏👏👏
Großartig👏
#gerührt❤️
👏👏👏👏
Love it
WAAAHNSINIG TOLL👏👏👏👏👏👏👏👏
👏👏👏👏👏👏gran idea
👏👏👏
Mega 🙌
👏
👏👏👏👏👏
👏
👏
👏
Spannendes Projekt
WOW 🙌👏
❤️
👍👍👍👍sehr spannend
👏
❤️
👏
👏
👌👌👍🖤💐💭
Tolle Idee🙌
🔥❤️
👏
💰/F
👏👏👏👏👏tolle Idee
🥀
🙌
🙌


Nextpost

👏
Super gemacht
👏
👏
Supergeil ❤️
👏👏👏
👏👏👏👏👏
❤️
👏👏
Stabil bruder💪💪🔥🔥
👏
👏
👏👏👏👏👏🏻
❤️
😊
Sophie lebt👏🔥🙌❤️
❤️
❤️
🔥❤️


Nextpost

XD
Respekt ❤️
❤️
❤️
🖤
...
Wunderbare Idee❤️
👍
👏
😍
❤️❤️❤️
👏
HAMMERIDEE🔥🔥🔥👏
❤️
Sehr gut
🌹
😍
👏👏👏
Suuuper gemacht👏👏
#idontcare
👏
Bin gespannt
❤️
👏👏
👏
👏
✊✊✊


Nextpost

😍
Super Projekt🔥
👏
Schizophrenia
Please explain
❤️
❤️
❤️
❤️❤️❤️❤️
❤️❤️❤️
👏❤️
❤️


Nextpost

Lol
😢🔥❤️
😂
❤️🙌
😮
❤️
👏
😢
❤️
👏
👏👏👏👏👏👏
❤️❤️
😍
😍😍😍
❤️❤️
🌞
👏👏👍👍
🙌
❤️❤️❤️
❤️❤️
❤️
❤️
❤️
❤️✊🌹
❤️


Nextpost

Sikikler sizi amk
@__joe__swanson__ der linke sieht so aus wie du
@__joe__swanson__ xaxaxa
Tolles Bild ❤️
❤️
@mayas1206
❤️
❤️
❤️
❤️
❤️
❤️
❤️
❤️
❤️
❤️
❤️
❤️
❤️
❤️
❤️
❤️
Tolle Idee 👍
👏
🙌
❤️❤️❤